# Get the Data


The NPI data file is too large to open using the usual methodology. Make use of the chunksize parameter in the panda's funtion to split the data into workable sizes. Use the columns data file to identify and parse out the columns that are needed. Finally, create separate tables for each subset of data and export them into a csv file. 

In [1]:
import pandas as pd
import numpy as np
KEY = 'NPI'

In [7]:
def get_file():
    reader = pd.read_csv("npidata_pfile_20050523-20211010.csv", chunksize=200000,low_memory=False,header=0)
    return reader
columns = pd.read_csv("npidata_pfile_20050523-20211010_FileHeader.csv")

In [9]:
taxonomy = columns.columns.str.contains("taxonomy", case=False)
taxonomy = columns.columns[taxonomy].values
main = columns.drop(columns=taxonomy)
taxonomy = np.append(taxonomy, KEY)

In [4]:
address = columns.columns.str.contains("address", case=False)
address = columns.columns[address].values
main = main.drop(columns=address)
address = np.append(address, KEY)

In [5]:
op = columns.columns.str.contains("other provider", case=False)
op = columns.columns[op].values
main = main.drop(columns=op)
op = np.append(op, KEY)

In [6]:
pl = columns.columns.str.contains("provider license", case=False)
pl = columns.columns[pl].values
main = main.drop(columns=pl)
pl = np.append(pl, KEY)

In [7]:
po = columns.columns.str.contains("provider other", case=False)
po = columns.columns[po].values
main = main.drop(columns=po)
po = np.append(po, KEY)

In [10]:
ao = columns.columns.str.contains("authorized official", case=False)
ao = columns.columns[ao].values
main = main.drop(columns=ao)
ao = np.append(ao, KEY)

In [9]:
parent = columns.columns.str.contains("parent", case=False)
parent = columns.columns[parent].values
main = main.drop(columns=parent).columns.values
parent = np.append(parent, KEY)

In [10]:
tablecolumns = [(main, "main"), (parent, "parent"), (taxonomy, "taxonomy"), (address, "address")]
for cols in tablecolumns:
    table = pd.concat([chunk[cols[0]] for chunk in get_file()], ignore_index=True)
    table.to_csv(cols[1] +".csv", index=False)



In [13]:
#Havign a look at the ao file.
ao_table = pd.concat([chunk[ao] for chunk in get_file()], ignore_index=True)
ao_table.to_csv("authorizedofficial.csv", index=False)

# Cleanup

In [245]:
address = pd.read_csv('address.csv', low_memory=False)
address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7054121 entries, 0 to 7054120
Data columns (total 17 columns):
 #   Column                                                                      Dtype 
---  ------                                                                      ----- 
 0   Provider First Line Business Mailing Address                                object
 1   Provider Second Line Business Mailing Address                               object
 2   Provider Business Mailing Address City Name                                 object
 3   Provider Business Mailing Address State Name                                object
 4   Provider Business Mailing Address Postal Code                               object
 5   Provider Business Mailing Address Country Code (If outside U.S.)            object
 6   Provider Business Mailing Address Telephone Number                          object
 7   Provider Business Mailing Address Fax Number                                object
 8   Pr

In [247]:
address_types = ["mailing", "location"]
new_names = ["First Line", "Second Line", "City", "State", "Postal Code", "Country Code", "Telephone", "Fax"]
col_change = {"NPI" : "NPI"}
i = 0
for address_type in address_types:
    cols = address.columns.str.contains(address_type, case=False)
    cols = address.columns[cols].values
    subset = new_names
    cols = np.append(cols, KEY)
    tbl = address[cols].reset_index(drop=True)
    for n in new_names:
        s = tbl.columns.str.contains(n) 
        s = tbl.columns[s].values
        col_change[s[0]] =  n
    tbl.rename(columns=col_change, inplace=True)
    tbl['Type'] = address_type
    tbl = tbl.dropna(subset = subset, how='all') 
    if i == 0:
        address_table = tbl
    else:
        address_table = pd.concat([address_table, tbl])
    i +=1
    
address_table.to_csv("cleaned_addressess.csv", index=False)

In [248]:
taxonomy = pd.read_csv('taxonomy.csv', low_memory=False)
taxonomy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7054121 entries, 0 to 7054120
Data columns (total 46 columns):
 #   Column                                          Dtype 
---  ------                                          ----- 
 0   Healthcare Provider Taxonomy Code_1             object
 1   Healthcare Provider Primary Taxonomy Switch_1   object
 2   Healthcare Provider Taxonomy Code_2             object
 3   Healthcare Provider Primary Taxonomy Switch_2   object
 4   Healthcare Provider Taxonomy Code_3             object
 5   Healthcare Provider Primary Taxonomy Switch_3   object
 6   Healthcare Provider Taxonomy Code_4             object
 7   Healthcare Provider Primary Taxonomy Switch_4   object
 8   Healthcare Provider Taxonomy Code_5             object
 9   Healthcare Provider Primary Taxonomy Switch_5   object
 10  Healthcare Provider Taxonomy Code_6             object
 11  Healthcare Provider Primary Taxonomy Switch_6   object
 12  Healthcare Provider Taxonomy Code_7       

In [249]:
titles = []
numbers = []
for cols in taxonomy.columns:
    title = cols.split("_")
    titles = np.append(titles, title[0])
    if len(title) == 2:
        numbers = np.append(numbers, title[1])
titles = np.unique(titles)
numbers = np.unique(numbers)
titles

array(['Healthcare Provider Primary Taxonomy Switch',
       'Healthcare Provider Taxonomy Code',
       'Healthcare Provider Taxonomy Group', 'NPI'], dtype='<U43')

In [250]:
numbers

array(['1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6',
       '7', '8', '9'], dtype='<U32')

In [251]:
subset = titles[titles != KEY]
subset

array(['Healthcare Provider Primary Taxonomy Switch',
       'Healthcare Provider Taxonomy Code',
       'Healthcare Provider Taxonomy Group'], dtype='<U43')

In [271]:
i = 0
cols = []
col_change = {"NPI" : "NPI"}
for number in numbers:
    print('working on taxonomy number', number)
    ##Here update this to get the column names 
    for title in titles:
        if title == KEY:
            cols = np.append(cols, KEY)
        else:
            cols = np.append(title+"_"+str(number), cols)
            col_change[title+"_"+str(number)] =  title
    
    ##Just grab those cols. 
    tbl = taxonomy[cols].reset_index(drop=True)
    tbl.rename(columns=col_change, inplace=True)
    tbl = tbl.dropna(subset=subset, how='all')
    if i == 0:
        taxonomy_table = tbl
    else:
        taxonomy_table = pd.concat([taxonomy_table, tbl])
    i +=1
    cols = []
taxonomy_table.to_csv("cleaned_taxonomy.csv", index=False)
taxonomy_table.head()

working on taxonomy number 1
working on taxonomy number 10
working on taxonomy number 11
working on taxonomy number 12
working on taxonomy number 13
working on taxonomy number 14
working on taxonomy number 15
working on taxonomy number 2
working on taxonomy number 3
working on taxonomy number 4
working on taxonomy number 5
working on taxonomy number 6
working on taxonomy number 7
working on taxonomy number 8
working on taxonomy number 9


,Healthcare Provider Taxonomy Group,Healthcare Provider Taxonomy Code,Healthcare Provider Primary Taxonomy Switch,NPI
0,NaN,207X00000X,Y,1679576722
1,NaN,207RC0000X,Y,1588667638
2,NaN,251G00000X,Y,1497758544
4,NaN,174400000X,N,1215930367
5,NaN,251G00000X,Y,1023011178


In [272]:
parents = pd.read_csv('parent.csv', low_memory=False)
parents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7054121 entries, 0 to 7054120
Data columns (total 3 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   Parent Organization LBN  object
 1   Parent Organization TIN  object
 2   NPI                      int64 
dtypes: int64(1), object(2)
memory usage: 161.5+ MB


In [273]:
columns = parents.columns.values
subset = columns[columns != KEY]
subset

array(['Parent Organization LBN', 'Parent Organization TIN'], dtype=object)

In [274]:
parent_table = parents.dropna(subset=subset, how="all").reset_index(drop=True)
parent_table.to_csv("cleaned_parent.csv", index=False)

In [3]:
endpoint = pd.read_csv("endpoint.csv", low_memory=False)
endpoint.head()

,NPI,Endpoint Type,Endpoint Type Description,Endpoint,Affiliation,Endpoint Description,Affiliation Legal Business Name,Use Code,Use Description,Other Use Description,Content Type,Content Description,Other Content Description,Affiliation Address Line One,Affiliation Address Line Two,Affiliation Address City,Affiliation Address State,Affiliation Address Country,Affiliation Address Postal Code
0,1154324382,DIRECT,Direct Messaging Address,rclose13800@MHSDIRECT.NET,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3501 Johnson St,NaN,Hollywood,FL,US,330215421
1,1962405175,DIRECT,Direct Messaging Address,fredericstelzer@epgi.allscriptsdirect.net,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1501 N Cedar Crest Blvd,suite 110,Allentown,PA,US,181042309
2,1699778894,DIRECT,Direct Messaging Address,aawomolo@direct.iuhealth.org,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2401 W University Ave,NaN,Muncie,IN,US,47303
3,1841293040,CONNECT,CONNECT URL,Jennifer.Giles_1@direct.cyfluentchart.com,N,Direct Email,NaN,HIE,Health Information Exchange (HIE),NaN,OTHER,Other,CMS esMD eMDR,31519 Winterplace Pkwy,Ste 1,Salisbury,MD,US,218041884
4,1306849591,DIRECT,Direct Messaging Address,svogel89246@direct.geisinger.edu,N,NaN,NaN,DIRECT,Direct,NaN,NaN,NaN,NaN,16 Woodbine Ln,NaN,Danville,PA,US,178218029


In [4]:
endpoint["Endpoint Type Description"].value_counts()

Direct Messaging Address    206372
CONNECT URL                  43537
Other URL                    31465
FHIR URL                     14606
SOAP URL                     10404
RESTful URL                    468
Name: Endpoint Type Description, dtype: int64

In [6]:
endpoint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306989 entries, 0 to 306988
Data columns (total 19 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   NPI                              306989 non-null  int64 
 1   Endpoint Type                    306852 non-null  object
 2   Endpoint Type Description        306852 non-null  object
 3   Endpoint                         303969 non-null  object
 4   Affiliation                      306989 non-null  object
 5   Endpoint Description             99127 non-null   object
 6   Affiliation Legal Business Name  33432 non-null   object
 7   Use Code                         150883 non-null  object
 8   Use Description                  150883 non-null  object
 9   Other Use Description            12185 non-null   object
 10  Content Type                     109971 non-null  object
 11  Content Description              109971 non-null  object
 12  Other Content De